## Connection

In [1]:
import os

from dotenv import load_dotenv
import numpy as np
import pandas as pd
from sqlalchemy import create_engine


# creds from .env file
load_dotenv()

POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_USERNAME = os.getenv('POSTGRES_USERNAME')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_DBNAME = os.getenv('POSTGRES_DBNAME')


# connection
try:
    conn = create_engine(f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    print('connected!')
except:
    print('Can`t establish connection to database')


# small function for selecting data
def select(sql):
    return pd.read_sql_query(sql, conn)

connected!


## UTM / Campaigns

In [2]:
sql = ''' 
select distinct utm_medium
from users;
'''

select(sql)

,utm_medium
0,cpc
1,


In [3]:
sql = ''' 
SELECT 
  utm_medium,
  COUNT(*) AS users_count,
  COUNT(CASE WHEN status = 'customer' THEN id END) AS customers_count
FROM users
GROUP BY 1
'''

select(sql)

,utm_medium,users_count,customers_count
0,cpc,499,134
1,,5583,640


Conversion rate - is basically a percentage of users who currently are customers.

In [4]:
sql = ''' 
WITH channel_stats AS (
  SELECT 
    utm_medium,
    COUNT(*) AS users_count,
    COUNT(CASE WHEN status = 'customer' THEN id END) AS customers_count
  FROM users
  GROUP BY 1
)

SELECT
  utm_medium,
  (1.0 * customers_count / users_count) AS customer_rate
FROM channel_stats
'''

select(sql)

,utm_medium,customer_rate
0,cpc,0.268537
1,,0.114634


In [8]:
sql = ''' 
select count(*)
from users 
where utm_source = 'twitter';

'''

select(sql)

,count
0,421


In [12]:
sql = ''' 
select 
    utm_campaign, 
    count(*)
from users
group by utm_campaign 
order by 2 desc
limit 2;
'''
select(sql)

,utm_campaign,count
0,,5583
1,20180208_us-ca-gb-au_mf_bindle-launch,300
